In [ ]:
import re
import jieba
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktLanguageVars

训练用文本最好事先分句，以便和录音对齐。如果没有已经分句的文本，也没有时间精力进行人工修订，可以尝试结合[结巴分词](https://github.com/fxsjy/jieba)和[NLTK](https://github.com/nltk/nltk)。

1. 继承`PunktLanguageVars`，自定义句结尾字符`sent_end_chars`
1. 先用jieba对文本进行一次分词，在词间加上空格。`PunktSentenceTokenizer`要求句结尾字符后面有空格，否则无法正确分句

In [ ]:
text = "音视频文件应只包含人声，音效、音乐等需事先另行去除，可使用音频编辑软件，如Audacity。文本文件应只包含录音的脚本，没有ID等额外信息，建议每句一行。成功安装和使用Aeneas需要espeak。"

class ChineseLanguageVars(PunktLanguageVars):
    sent_end_chars = ('。', '！', '？', '”')

tokenizer = PunktSentenceTokenizer(lang_vars=ChineseLanguageVars())
sentences = tokenizer.tokenize(' '.join(jieba.cut(text)))

可以进一步对句子进行简单分析。

* 目前的中文自定义声音模型暂不支持双语，最好去掉含英文词的句子，否则会影响模型质量。
* 句子长度一般用词数衡量，而对中文而言字数更合适，也可进行统计以供参考。

In [ ]:
for sentence in sentences:
    words = sentence.split(' ')
    sentence_len = 0
    for word in words:
        if re.compile('[a-zA-Z0-9]+').search(word):
            length = 1
            if re.compile('[a-zA-Z]+').search(word):
                print('Western word "{word}" detected in "{sentence}"'.format(word=word, sentence=sentence))
        elif re.compile('[。！？”、“‘’；：]+').search(word):
            length = 0
        else:
            length = len(word)
        sentence_len += length